<a href="https://colab.research.google.com/github/KARIMULLA-77/ANPR/blob/main/VEHICLE_NUMBER_PLATE_EXTRACTION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install easyocr

In [ ]:
pip install roboflow

In [ ]:
import cv2
import easyocr
import re
import pandas as pd
from roboflow import Roboflow
import os

API_KEY = "dnQosvZ7WHVcIZwHT4lO"
MODEL_ID = "auto-htksu/1"

rf = Roboflow(api_key=API_KEY)
project = rf.workspace().project(MODEL_ID.split('/')[0])
model = project.version(MODEL_ID.split('/')[1]).model

reader = easyocr.Reader(['en'])


plate_pattern = r'^[A-Z]{2}[0-9]{1,2}[A-Z]{1,2}[0-9]{3,4}$'

results_dir = "results"
os.makedirs(results_dir, exist_ok=True)

output_frames_dir = os.path.join(results_dir, "frames_with_bounding_boxes")
os.makedirs(output_frames_dir, exist_ok=True)

output_cropped_plates_dir = os.path.join(results_dir, "cropped_number_plates")
os.makedirs(output_cropped_plates_dir, exist_ok=True)

seen_plates = set()

video_path = "vid.mp4"
cap = cv2.VideoCapture(video_path)

results = []
frame_id = 0

fps = cap.get(cv2.CAP_PROP_FPS)
if fps > 0:
    frame_interval = int(fps * 1)
else:
    frame_interval = 1


while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_id += 1

    if frame_id % 5 != 0:
        continue


    temp_file = "temp_frame.jpg"
    cv2.imwrite(temp_file, frame)


    predictions = model.predict(temp_file, confidence=40, overlap=30).json()

    frame_with_boxes = frame.copy()
    for pred in predictions["predictions"]:
        vehicle_type = pred["class"]
        x, y, w, h = pred["x"], pred["y"], pred["width"], pred["height"]
        x1, y1, x2, y2 = int(x - w/2), int(y - h/2), int(x + w/2), int(y + w/2)

        crop = frame[y1:y2, x1:x2]

        ocr_result = reader.readtext(crop)

        for _, text, _ in ocr_result:
            candidate = text.upper().replace(" ", "")
            if re.fullmatch(plate_pattern, candidate) and candidate not in seen_plates:
                print(f"Frame {frame_id} | Vehicle: {vehicle_type} | Plate: {candidate}")
                results.append([frame_id, vehicle_type, candidate])
                seen_plates.add(candidate)
                plate_img_name = os.path.join(output_cropped_plates_dir, f"frame_{frame_id}_{candidate}.jpg")
                cv2.imwrite(plate_img_name, crop)
                cv2.rectangle(frame_with_boxes, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame_with_boxes, candidate, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    frame_with_boxes_name = os.path.join(output_frames_dir, f"frame_{frame_id}.jpg")
    cv2.imwrite(frame_with_boxes_name, frame_with_boxes)


cap.release()
excel_output_path = os.path.join(results_dir, "vehicle_type_with_number_plate.xlsx")
df = pd.DataFrame(results, columns=["Frame", "Vehicle Type", "Number Plate"])
df.to_excel(excel_output_path, index=False)

print(f"✅ Done! Results stored in {excel_output_path}")